<a href="https://colab.research.google.com/github/leidysguerrero120/Trabajo-Anal-tica-3---Marketing/blob/main/Exploraci%C3%B3n_y_limpieza_Marketing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sqlalchemy
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 35.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import plotly.express as px
from sqlalchemy import create_engine
import urllib.request


*** BASE DE DATOS EN SQL***

In [3]:
conn = sqlite3.connect('/content/drive/MyDrive/db_movies_final')

cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

for table in tables:
    print(table[0])

ratings
movies
usuarios_sel
movies_sel
ratings_final
movies_final
full_ratings


In [4]:
df_ratings = pd.read_sql_query("SELECT * FROM ratings LIMIT 5;", conn)
print("Primeras filas de la tabla 'ratings':")
print(df_ratings)

df_movies = pd.read_sql_query("SELECT * FROM movies LIMIT 5;", conn)
print("\nPrimeras filas de la tabla 'movies':")
print(df_movies)


Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


In [5]:
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT
year: INTEGER


In [6]:
cur.execute("PRAGMA table_info(ratings)")
columns_ratings = cur.fetchall()

print("Tipos de datos de las columnas en la tabla 'ratings':")
for column in columns_ratings:
    print(column[1] + ": " + column[2])

cur.execute("PRAGMA table_info(movies)")
columns_movies = cur.fetchall()

print("\nTipos de datos de las columnas en la tabla 'movies':")
for column in columns_movies:
    print(column[1] + ": " + column[2])

Tipos de datos de las columnas en la tabla 'ratings':
userId: INTEGER
movieId: INTEGER
rating: REAL
timestamp: INTEGER

Tipos de datos de las columnas en la tabla 'movies':
movieId: INTEGER
title: TEXT
genres: TEXT
year: INTEGER


In [7]:
query = f"SELECT * FROM {'ratings'}"

dfr = pd.read_sql_query(query, conn)

print("Primeras filas de la tabla '{}':".format('ratings'))
print(dfr.head())

Primeras filas de la tabla 'ratings':
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [8]:
query = f"SELECT * FROM {'movies'}"

dfm = pd.read_sql_query(query, conn)

print("Primeras filas de la tabla '{}':".format('movies'))
print(dfm.head())

Primeras filas de la tabla 'movies':
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  


In [9]:
print("\nCampos de cruce:")
print("Tabla 'ratings':", dfr.columns)
print("Tabla 'movies':", dfm.columns)

print("\nFormato de 'movieId':")
print("Tabla 'ratings':", dfr['movieId'].dtype)
print("Tabla 'movies':", dfm['movieId'].dtype)

duplicates_ratings = dfr.duplicated().sum()
duplicates_movies = dfm.duplicated().sum()

print("\nCantidad de duplicados en la tabla 'ratings':", duplicates_ratings)
print("Cantidad de duplicados en la tabla 'movies':", duplicates_movies)


Campos de cruce:
Tabla 'ratings': Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
Tabla 'movies': Index(['movieId', 'title', 'genres', 'year'], dtype='object')

Formato de 'movieId':
Tabla 'ratings': int64
Tabla 'movies': int64

Cantidad de duplicados en la tabla 'ratings': 0
Cantidad de duplicados en la tabla 'movies': 0


RATINGS

***Distribución de las calificaciones***

In [10]:
query = """
        SELECT
            rating,
            COUNT(*) AS conteo
        FROM
            ratings
        GROUP BY
            rating
        ORDER BY
            conteo DESC
        """

dfr2 = pd.read_sql_query(query, conn)

print(dfr2)



   rating  conteo
0     4.0   26818
1     3.0   20047
2     5.0   13211
3     3.5   13136
4     4.5    8551
5     2.0    7551
6     2.5    5550
7     1.0    2811
8     1.5    1791
9     0.5    1370


In [11]:
import plotly.graph_objects as go

data = go.Bar(
    x=dfr2.rating,
    y=dfr2.conteo,
    text=dfr2.conteo,
    textposition="outside",
    marker=dict(
        color=dfr2.rating,
        colorscale='Viridis',
        showscale=False
    ),
    hoverinfo='x+y',
)


layout = go.Layout(
    title={
        'text': "Conteo de calificaciones",
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=24, color='darkblue'),
    xaxis=dict(
        title='Rating',
        tickmode='linear',
        tickfont=dict(size=14, color='darkgray')
    ),
    yaxis=dict(
        title='Count',
        tickfont=dict(size=14, color='darkgray')
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(255,255,255,0.8)',
    margin=dict(l=40, r=40, t=80, b=40),
    hovermode='closest',
)


fig = go.Figure(data=data, layout=layout)


fig.show()


El rating de 4.0 destaca como el más popular, acumulando un total de 26,818 calificaciones. Este alto número sugiere que una gran parte de los usuarios ha tenido experiencias satisfactorias con las películas recomendadas, lo que es un indicador positivo de la calidad del contenido ofrecido.

Por otro lado, el rating más bajo, 0.5, ha recibido solo 1,370 calificaciones. Esta cifra relativamente baja indica que, aunque hay usuarios insatisfechos, su número es limitado. Sin embargo, estos ratings negativos pueden tener un impacto significativo en la percepción general de la plataforma, ya que las experiencias negativas suelen resonar más entre los usuarios potenciales.

Para abordar esta problemática, es crucial que el equipo de desarrollo del sistema de recomendaciones no solo identifique y promueva las características de las películas que contribuyen a la calificación de 4.0, sino que también preste atención a las razones detrás de las calificaciones más bajas. Esto permitirá mejorar el contenido ofrecido y adaptar las recomendaciones, asegurando así una experiencia más satisfactoria y fomentando la fidelización de los usuarios.

In [12]:
rating_users = pd.read_sql('''SELECT userId as user_id,
                                     COUNT(*) as cnt_rat
                              FROM ratings
                              GROUP BY userId
                              ORDER BY cnt_rat DESC''', conn)

# Imprimir el DataFrame
print(rating_users)

     user_id  cnt_rat
0        414     2698
1        599     2478
2        474     2108
3        448     1864
4        274     1346
..       ...      ...
605      207       20
606      194       20
607      189       20
608      147       20
609       53       20

[610 rows x 2 columns]


En el análisis de la actividad de calificación de los usuarios, se observa que el usuario 414 es el más activo, con un total impresionante de 2,698 calificaciones de películas. Este elevado número de calificaciones no solo indica un fuerte compromiso y participación con la plataforma, sino que también sugiere que este usuario tiene una amplia experiencia con el contenido disponible. La diversidad de películas que ha calificado puede ofrecer una valiosa fuente de datos para el sistema de recomendaciones, ya que su historial puede ayudar a identificar patrones de gustos que podrían ser extrapolables a otros usuarios con preferencias similares. Aprovechar este tipo de usuarios activos para enriquecer las recomendaciones podría ser clave para mejorar la experiencia general.

Por el contrario, el usuario 53 ha calificado únicamente 20 películas, lo que lo convierte en el usuario menos activo en términos de calificación. Esta disparidad en la actividad de calificación podría señalar varios factores, como un menor interés en el contenido disponible o una falta de interacción con la plataforma. Este tipo de usuarios, que tienen una participación mínima, representan un reto para la plataforma, ya que su escasa actividad puede dificultar la recopilación de datos significativos para ofrecer recomendaciones personalizadas. Para abordar esta problemática, es esencial que el sistema de recomendaciones no solo se centre en usuarios altamente activos como el 414, sino que también busque maneras de incentivar a los usuarios menos activos a interactuar más con la plataforma, ya sea mediante notificaciones, contenido relevante o mejoras en la interfaz de usuario que faciliten la calificación de películas. Esto podría ayudar a equilibrar la actividad de calificación y optimizar la calidad de las recomendaciones proporcionadas.

In [13]:
import plotly.express as px

fig = px.histogram(
    rating_users,
    x='cnt_rat',
    title='Histograma de la Frecuencia del Número de Calificaciones por Usuario',
    template='plotly_white',
    color_discrete_sequence=['#4C78A8'],
)


fig.update_traces(marker=dict(line=dict(width=1, color='darkgray')))


fig.update_layout(
    title_x=0.5,
    title_font=dict(size=24, color='darkblue'),
    xaxis_title='Número de Calificaciones',
    yaxis_title='Frecuencia',
    margin=dict(l=40, r=40, t=80, b=40)
)


fig.show()


In [14]:
rating_users.describe()

,user_id,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


**Resumen Estadístico**

1. **Total de usuarios analizados:**610

2. **Número promedio de calificaciones por usuario:**165.3. Este promedio sugiere un nivel moderado de interacción, donde los usuarios parecen involucrarse con el contenido disponible, pero también indica que hay espacio para mejorar la participación, especialmente entre los usuarios menos activos.

3. **Desviación estándar:**269.5, lo que señala una variabilidad considerable en el número de calificaciones. Algunos usuarios califican muchas películas, mientras que otros son menos activos, lo que puede dificultar la personalización de las recomendaciones.

**Distribución de Calificaciones**

1. **Número mínimo de calificaciones:** 20. Este dato refleja que hay usuarios que han calificado muy pocas películas, lo que puede sugerir falta de interés o dificultad para encontrar contenido relevante.

2. **Número máximo de calificaciones:** 2698. Este número resalta la existencia de usuarios extremadamente activos que han calificado un amplio rango de películas, indicando una alta lealtad a la plataforma.

**Percentiles**

1. **Percentil 25 (Q1):** 35 calificaciones. Esto significa que el 25% de los usuarios ha calificado 35 películas o menos, lo que indica que una porción significativa de la base de usuarios puede no estar suficientemente comprometida.

2. **Mediana (50%):** 70.5 calificaciones. Este valor implica que la mitad de los usuarios ha calificado 70 películas o menos, sugiriendo que una buena parte de los usuarios no está maximizando su interacción con la plataforma.

3. **Percentil 75 (Q3):** 168 calificaciones. Esto indica que el 75% de los usuarios ha calificado 168 películas o menos, sugiriendo que solo una pequeña proporción de usuarios está calificando más de esa cantidad.


Las implicaciones para la plataforma incluyen la necesidad de implementar recomendaciones personalizadas y crear incentivos para aumentar la participación de usuarios menos activos, mejorando así la experiencia general y fomentando la fidelización y el compromiso con el contenido.

In [15]:
query = '''
    SELECT userId as user_id,
           COUNT(*) as cnt_rat
    FROM ratings
    GROUP BY userId
    HAVING cnt_rat >= 50 AND cnt_rat <= 1000
    ORDER BY cnt_rat DESC
'''
rating_users2 = pd.read_sql(query, conn)

print(rating_users2)

     user_id  cnt_rat
0        182      977
1        307      975
2        603      943
3        298      939
4        177      904
..       ...      ...
368      457       50
369      237       50
370      170       50
371      124       50
372       31       50

[373 rows x 2 columns]


En el nuevo dataframe, el usuario con mayor cantidad de calificaciones es el 182, con un total de 977 calificaciones. Este nivel de actividad indica un fuerte compromiso con la plataforma, lo que puede sugerir una satisfacción alta con las recomendaciones y el contenido disponible.

Por otro lado, varios usuarios, incluyendo 31, 124, 170 y 237, han realizado solo 50 calificaciones cada uno. Esta baja actividad puede señalar que estos usuarios no están completamente comprometidos o que necesitan una mayor motivación para explorar más películas. Este contraste entre usuarios altamente activos y aquellos con pocas calificaciones resalta la importancia de implementar estrategias que fomenten la participación de todos los usuarios, especialmente los menos activos.

In [16]:
rating_users2.describe()

,user_id,cnt_rat
count,373.000000,373.000000
mean,301.766756,201.863271
std,178.462796,187.219913
min,1.000000,50.000000
25%,141.000000,77.000000
50%,305.000000,129.000000
75%,455.000000,248.000000
max,608.000000,977.000000


**Resumen de Estadísticas:**

1. **Total de usuarios analizados:** 373

2. **Promedio de calificaciones por usuario:** 201.86. Esto sugiere que, en promedio, los usuarios han calificado alrededor de 202 películas, lo cual refleja un compromiso moderado con la plataforma.

3. **Desviación estándar: **187.22, indicando una gran variabilidad entre los usuarios, donde algunos califican muchas películas mientras que otros lo hacen de manera limitada.

**Distribución de Calificaciones**

1. **Número mínimo de calificaciones:** 50. Este es el valor más bajo, lo que sugiere que hay un umbral mínimo de calificaciones en los usuarios menos activos.

2. **Número máximo de calificaciones:** 977, lo que muestra que el usuario más comprometido ha calificado casi 1000 películas.

**Percentiles**

1. **Percentil 25 (Q1):** 77 calificaciones, lo que significa que el 25% de los usuarios ha calificado 77 películas o menos.

2. **Mediana (50%):** 129 calificaciones, lo que indica que la mitad de los usuarios ha calificado hasta 129 películas.

3. **Percentil 75 (Q3):** 248 calificaciones, lo que indica que el 75% de los usuarios ha calificado hasta 248 películas.

Este análisis indica que, aunque algunos usuarios son altamente activos, una porción significativa de usuarios tiene una menor interacción con la plataforma. Es crucial implementar estrategias de recomendación personalizadas para motivar a los usuarios menos activos y mejorar la experiencia general en la plataforma.

In [17]:
import plotly.express as px


fig = px.histogram(
    rating_users2,
    x='cnt_rat',
    title='Histograma de Frecuencia del Número de Calificaciones por Usuario',
    labels={'cnt_rat': 'Número de Calificaciones'},
    color_discrete_sequence=['#2E91E5']
)


fig.update_layout(
    title={'x':0.5, 'xanchor': 'center'},
    xaxis_title='Número de Calificaciones',
    yaxis_title='Frecuencia',
    bargap=0.1,
    template='plotly_white'
)


fig.show()


El análisis del histograma revela una clara concentración de usuarios que han calificado entre 50 y 99 películas, con un total de 137 usuarios en ese rango. Esto indica que una gran parte de los usuarios interactúan con la plataforma a un nivel moderado. A continuación, le siguen 72 usuarios que han calificado entre 100 y 149 películas, lo que demuestra una disminución progresiva en la actividad de calificación a medida que aumenta el número de películas evaluadas.

Este patrón sugiere que la mayoría de los usuarios realiza un número limitado de calificaciones, lo cual es un indicador clave para mejorar las estrategias de recomendación y motivación en la plataforma.

**Distribución de calificaciones por película**

In [18]:
query = """
        SELECT
            movieId,
            COUNT(*) AS cnt_rat
        FROM
            ratings
        GROUP BY
            movieId
        ORDER BY
            cnt_rat DESC
        """

rating_movies = pd.read_sql_query(query, conn)

print(rating_movies)

      movieId  cnt_rat
0         356      329
1         318      317
2         296      307
3         593      279
4        2571      278
...       ...      ...
9719       96        1
9720       83        1
9721       77        1
9722       55        1
9723       49        1

[9724 rows x 2 columns]


La película con mayor cantidad de calificaciones es la identificada con el ID 356, acumulando un total de 329 calificaciones. Este alto nivel de interacción indica que es una de las películas más vistas o más populares en la plataforma, lo que podría sugerir su relevancia para futuros algoritmos de recomendación. Además, analizar las características de esta película podría proporcionar información valiosa para identificar patrones en las preferencias de los usuarios.

In [19]:
rating_movies.describe()

,movieId,cnt_rat
count,9724.000000,9724.000000
mean,42245.024373,10.369807
std,52191.137320,22.401005
min,1.000000,1.000000
25%,3245.500000,1.000000
50%,7300.000000,3.000000
75%,76739.250000,9.000000
max,193609.000000,329.000000


In [21]:
import plotly.express as px


fig = px.histogram(
    rating_movies,
    x='cnt_rat',
    title='Frecuencia de calificaciones por película',
    nbins=50,
    labels={'cnt_rat': 'Número de calificaciones'},
    color_discrete_sequence=['#636EFA']
)


fig.update_layout(
    xaxis_title='Número de calificaciones por película',
    yaxis_title='Frecuencia',
    bargap=0.2,
    template='plotly_dark',
    title_x=0.5
)

fig.show()


In [22]:
###Creamos un dataframe que contenga solo películas cuya cantidad de calificaciones sea igual o mayor que 50.
###Filtramos las películas que tengan 50 o más calificaciones

rating_movies2 = pd.read_sql('''
    SELECT movieId, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY movieId
    HAVING cnt_rat >= 50
    ORDER BY cnt_rat DESC
''', conn)

In [23]:
rating_movies2.describe()

,movieId,cnt_rat
count,450.000000,450.000000
mean,11245.017778,91.911111
std,23457.084170,46.098385
min,1.000000,50.000000
25%,757.000000,59.000000
50%,2020.000000,76.000000
75%,4994.500000,107.750000
max,122904.000000,329.000000


El análisis muestra que después de aplicar un filtro de películas con al menos 50 calificaciones, el total es de 450 películas . Estas películas tienen un promedio de 92 calificaciones cada una.

El número mínimo de calificaciones es 50 y el máximo es 329 .
En cuanto a los percentiles:
El 25% de las películas tiene 59 calificaciones o menos .
El 50% tiene 76 calificaciones o menos .
Esto indica que, aunque se ha filtrado el conjunto de películas, sigue habiendo una concentración significativa de películas con un número relativamente bajo de calificaciones, lo que sugiere que la plataforma podría tener una distribución de popularidad desigual. Algunas reciben muchas películas, mientras que muchas otras se quedan cerca del umbral mínimo de 50.

In [26]:
import plotly.express as px


fig = px.histogram(
    rating_movies2,
    x='cnt_rat',
    title='Frecuencia de número de calificaciones por película',
    color_discrete_sequence=['#636EFA'],
    template='plotly_dark'
)

fig.update_layout(
    xaxis_title='Número de Calificaciones',
    yaxis_title='Frecuencia',
    margin=dict(l=50, r=50, t=100, b=50),
    title_x=0.5
)

fig.update_traces(
    marker=dict(
        line=dict(width=2, color='white')
    )
)


fig.show()



El histograma refleja una clara concentración de películas con un número moderado de calificaciones. Específicamente, 115 películas han sido calificadas entre 50 y 59 veces, lo que representa el grupo más numeroso. A continuación, 76 películas tienen entre 60 y 69 calificaciones. Curiosamente, solo una película ha recibido entre 320 y 329 calificaciones, lo que resalta una notable desigualdad en la distribución de las calificaciones. Esto sugiere que la mayoría de las películas tienen pocas calificaciones, mientras que muy pocas obtienen un alto número de opiniones.

In [30]:
###Género con mejor promedio de calificación

conn = sqlite3.connect('/content/drive/MyDrive/db_movies_final')

query = """
        SELECT movies_final.genres, AVG(full_ratings.rating) AS avg_rating
        FROM full_ratings
        JOIN movies_final ON full_ratings.movie_id = movies_final.movie_id
        GROUP BY movies_final.genres
        ORDER BY avg_rating DESC;
        """
cursor = conn.cursor()
cursor.execute(query)

result = pd.read_sql(query, conn)

print(result.head)

conn.close()

<bound method NDFrame.head of                                                 genres  avg_rating
0                                           Comedy|War    4.266304
1              Action|Adventure|Comedy|Fantasy|Romance    4.246032
2                               Crime|Mystery|Thriller    4.200348
3                             Action|Adventure|Western    4.185484
4                                            Drama|War    4.167183
..                                                 ...         ...
196  Adventure|Animation|Children|Comedy|Fantasy|Sc...    2.744681
197                      Comedy|Fantasy|Romance|Sci-Fi    2.681159
198                       Action|Comedy|Crime|Thriller    2.630000
199                                Action|Crime|Sci-Fi    2.616071
200                       Action|Comedy|Sci-Fi|Western    2.230769

[201 rows x 2 columns]>


El análisis de géneros revela que el género **Action|Adventure|Comedy|Fantasy|Romance** tiene el promedio más alto de calificaciones, con una puntuación de **4.24**, lo que indica una fuerte preferencia del público hacia películas que combinan acción, aventura, comedia, fantasía y romance. Esto sugiere que las películas dentro de este género logran capturar la atención de una audiencia diversa y logran mantener un alto nivel de satisfacción.

Por otro lado, el género con el peor promedio de calificaciones es **Action|Comedy|Sci-Fi|Western**, con una puntuación de **2.13**, lo que puede indicar que este tipo de películas no logra conectar de la misma manera con los espectadores o que su combinación de elementos de acción, comedia, ciencia ficción y western no tiene la misma atracción. Esta diferencia sugiere que algunos géneros mixtos pueden resultar más atractivos que otros, posiblemente debido a la dificultad de equilibrar los elementos narrativos.